In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
from tqdm.notebook import tqdm
import pandas as pd
import ast
import re
import urllib.parse
import json


In [7]:
hipcamp_market_listings = {
    "Texas Hill Country, TX": "https://www.hipcamp.com/en-US/search/?arrive=2022-05-14&depart=2022-05-15&sw_lat=30.57526592670365&sw_lng=-97.60344623249766&ne_lat=33.7688304669544&ne_lng=-94.63804007557152"
}

test_url = hipcamp_market_listings.get("Texas Hill Country, TX")
answer = requests.get(test_url, timeout=500)
content = answer.content
soup = BeautifulSoup(content, "html.parser")

In [20]:

outlist = []
for i in hipcamp_market_listings.keys():
    print(i)
    url_test = hipcamp_market_listings.get(i)
    answer = requests.get(url_test, timeout=500)
    content = answer.content
    soup = BeautifulSoup(content, "html.parser")
    counter = 0
    for x in soup.find_all('script'):
        try:
            subpayload = re.sub('[\[CDATA\]]', '', x.string)
        except TypeError:
            continue
        if "discover_payload=" in subpayload:        
            print("________", counter)
            payload = subpayload.split("discover_payload=")[1]
            #print (payload)
        counter += 1
    start_string = '"accom_icons"'
    end_string = '"verified_tooltip_text":'
    result = re.findall(f'{start_string}(.*?){end_string}', payload)
    print(type(result))
    count = 1
    for r in result:
        output = "{" + start_string + r + "}"
        output = output.replace('true', '1').replace('false', '0')
        output = output.replace('"tent","rv"','["tent","rv"]')        
        output0 = output.split('"top_photos"')[0]
        output1 = '"url"' + output.split('"url"')[1]
        o = output0 + output1
        #print(payload)
        #break
        #o = o.encode().decode('unicode-escape')
        #o = re.sub(r"\\\\",r" ",o)
        #o = o.replace("\'", "\"")
        o = o.replace('null', '"None"')
        try:
            d = ast.literal_eval(o)
            d["market"] = i
            outlist.append(d)
            #print("success", count)
        except:
            pass        
        count += 1
    

Texas Hill Country, TX
________ 3
<class 'list'>
{"hipcamp_campgrounds":{"accom_icons":"tent","rv","bookings_count":587,"campsites_count":10,"distance_to_search_place":null,"favorites_count":723,"has_enough_recommends":true,"id":18144,"insertion_id":"c9p9ulta0luuv263mur0","is_highly_demanded":true,"is_instant_bookable":true,"is_newly_live":false,"is_price_discount":false,"lat":"32.262511933951722","lng":"-96.202619590429076","name":"Llamaland ent amping","owner_first_name":"Judy","park_name":"Llama Land Ranch","park_url":"/en-US/discover/texas/llama-land-ranch","price_per_night":30,"price_per_night_formatted":"$30","price_per_night_base_formatted":"$30","recommends_count":361,"recommends_percentage":99.8614958448753,"top_photos":{"uuid":"426256ce-9e4f-4d59-8a14-a7a71632071b","carousel_small":"https://hipcamp-res.cloudinary.com/images/c_fill,f_auto,g_auto,h_400,q_60,w_780/v1560883205/campground-photos/hg2hjwo9otkonjdqjn98/llama-land-ranch-llamaland-tent-camping.jpg"},{"uuid":"0b354dfc-8

In [17]:
o

'{"accom_icons":"house","bookings_count":2,"campsites_count":1,"distance_to_search_place":"None","favorites_count":3,"has_enough_recommends":0,"id":65543,"insertion_id":"c9p9rie7pepgp24ncha0","is_highly_demanded":0,"is_instant_bookable":1,"is_newly_live":0,"is_price_discount":0,"lat":"33.091","lng":"-94.9158","name":"2021 Hartland RV","owner_first_name":"Lynda","park_name":"oppergate Ranch","park_url":"/en-US/discover/texas/coppergate-ranch","price_per_night":100,"price_per_night_formatted":"$100","price_per_night_base_formatted":"$100","recommends_count":0,"recommends_percentage":0.0,"url":"/en-US/texas/coppergate-ranch/2021-hartland-rv","state_abbrv_name":"X","state_name":"exas","city_name":"Mount Pleasant","country_code":"US","is_verified":0,}'

In [15]:
df = pd.DataFrame(outlist)

In [16]:
df.head()

,accom_icons,bookings_count,campsites_count,distance_to_search_place,favorites_count,has_enough_recommends,id,insertion_id,is_highly_demanded,is_instant_bookable,...,price_per_night_base_formatted,recommends_count,recommends_percentage,url,state_abbrv_name,state_name,city_name,country_code,is_verified,market
0,"[tent, rv]",587,10,None,723,1,18144,c9p9rie7pepgp24ncem0,1,1,...,$30,361,99.861496,/en-US/texas/llama-land-ranch/llamaland-tent-c...,X,exas,ool,US,1,"Texas Hill Country, TX"
1,"[tent, rv]",856,14,None,884,1,36018,c9p9rie7pepgp24ncemg,1,0,...,$30,478,97.175732,/en-US/texas/4r-ranch-winery-campsite/4r-ranch...,X,exas,Muenster,US,1,"Texas Hill Country, TX"
2,tent,146,5,None,377,1,60687,c9p9rie7pepgp24ncen0,0,1,...,$40,79,96.202532,/en-US/texas/leez-beez-honeybee-haven/pollinat...,X,exas,Waxahachie,US,1,"Texas Hill Country, TX"
3,rv,242,4,None,632,1,49473,c9p9rie7pepgp24nceng,1,1,...,$45,134,98.507463,/en-US/texas/big-rocks-on-the-brazos/big-rocks...,X,exas,Waco,US,1,"Texas Hill Country, TX"
4,"[tent, rv]",64,1,None,201,1,48681,c9p9rie7pepgp24nceo0,0,1,...,$44,46,100.000000,/en-US/texas/daffodil-hill-retreat/daffodil-hi...,X,exas,,US,1,"Texas Hill Country, TX"


In [11]:
data_cols = ["campsites_count", "bookings_count", "price_per_night_formatted", "price_per_night_base_formatted"]

for i in data_cols:
    df[i] = df[i].replace("Studio", "0")
    df[i] = df[i].astype(str).str.extract("(\d+)")
    df[i] = pd.to_numeric(df[i], errors="coerce")


In [12]:
df["url"] = "https://www.hipcamp.com" + df["url"]

In [13]:
df.to_csv("hipcamp-glamping.csv")


In [23]:
# try for one page
#base_features = process_search_pages(url_list[4:5])
